# Replicating Qubit Results

This notebook attempts to replicate results from this [paper](https://www.science.org/doi/10.1126/sciadv.abl6698).

First problem: when specifying Ω, α, the values calculated by Transmon() are different and use the eigenenergies. Find a way to reverse-engineer the Hamiltonian so they are kept the same as specified.

In [ ]:
# imports
import pandas as pd
import numpy as np

from transmon_code.superoperators import *
from transmon_code.transmon import Transmon
from transmon_code.optimise_X90 import *
from transmon_code.simulate import *
from transmon_code.helpers import *

In [ ]:
# read data from csv fild
data = pd.read_csv("qubit_data.csv")

transmons = [Transmon(n_levels=8, Ω=2*np.pi*data["Ω/2π (MHz)"][i], α=2*np.pi*data["α/2π (MHz)"][i], RWA=False, t_decay=data["T1 (μs)"][i], t_dephase=data["T2 (μs)"][i], dt=0.00002) for i in range(len(data))]

# first one is the width between A=(A/e)
# second is zero-to-zero width
pulse_width = 0.04973593124638
pulse_width = 0.024

### TEST BELOW

transmons[0].X90_args = {'A': 152.76901236246255, 'Γ': 0.024, 'ω': 25007.275767054834, 'A_DRAG': -0.0013677941456950009}

transmons[1].X90_args = {'A': 152.80570762438495, 'Γ': 0.024, 'ω': 25409.587896997888, 'A_DRAG': -0.0011863850682256394}

transmons[2].X90_args = {'A': 152.88720727559814, 'Γ': 0.024, 'ω': 25943.997177293375, 'A_DRAG': -0.0007523755046243693}

transmons[3].X90_args = {'A': 152.9486513654043, 'Γ': 0.024, 'ω': 26333.12122293356, 'A_DRAG': -0.0011984655445269122}

data

In [ ]:
semiranges = {"A":40, "ω": 50, "Γ":0, "A_DRAG":0.05}

t = np.arange(0, transmons[0].X90_args["Γ"], transmons[0].dt)

for qubit_no in range(4):
    print("Qubit "+str(qubit_no))    

    transmons[qubit_no].t_decay = np.inf
    transmons[qubit_no].t_dephase = np.inf

    # apparently DRAG parameter is 0.5/α to maximise avg gate fidelity and 1/α to minimise leakage errors?

    transmons[qubit_no].X90_args = create_X90_pulse(t, transmons[qubit_no], transmons[qubit_no].X90_args, semiranges=semiranges, plot=True, N=40) 

for i,v in enumerate(transmons):
    print()
    print("Transmon "+str(i))
    print(v.X90_args)
    

In [ ]:
# simulates pulse and breaks down error

# %matplotlib qt

for qubit_no in range(4):

    # transmons[qubit_no].t_decay = data["T1 (μs)"][qubit_no]
    # transmons[qubit_no].t_dephase = data["T2 (μs)"][qubit_no]

    tmp_args = deepcopy(transmons[qubit_no].X90_args)
    # tmp_args["A_DRAG"] = 0

    targ = (basis(transmons[qubit_no].n_levels, 0) - 1j*basis(transmons[qubit_no].n_levels, 1)).unit()

    res, f = simulate(transmons[qubit_no], tmp_args, target=targ, plot=True)

    break_down_errors(transmons[qubit_no], tmp_args, res[-1], f)